In [ ]:
!apt-get install ffmpeg
import os
os.environ["CUDA_HOME"] = "/usr/local/cuda-11"
!echo $CUDA_HOME
!git clone https://github.com/cedro3/iPERCore.git
%cd /content/iPERCore/


In [ ]:
#Setup
#!python setup.py develop
! pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.1 -f https://download.pytorch.org/whl/torch_stable.html
! pip install mmcv-full==1.2.2
! pip install git+https://github.com/open-mmlab/mmdetection.git@8179440ec5f75fe95484854af61ce6f6279f3bbc
! pip install git+https://github.com/open-mmlab/mmediting@d4086aaf8a36ae830f1714aad585900d24ad1156
! pip install git+https://github.com/iPERDance/neural_renderer.git@e5f54f71a8941acf372514eb92e289872f272653
! pip install tensorboardX>=2.1

In [ ]:
#download weight
#! pip install --upgrade gdown
import gdown
gdown.download('https://drive.google.com/uc?id=1jpp_KytMplNNFA_IJSzzSWhvyqIdmnJv', 'assets/checkpoints.zip', quiet=False)
!unzip -o assets/checkpoints.zip -d assets/
!rm assets/checkpoints.zip
!wget -O assets/samples.zip  "https://download.impersonator.org/iper_plus_plus_latest_samples.zip"
!unzip -o assets/samples.zip -d  assets
!rm assets/samples.zip

In [ ]:
%cd /content/iPERCore/

import os
import os.path as osp
import platform
import argparse
import time
import sys
import subprocess
from IPython.display import HTML
from base64 import b64encode


In [ ]:
# the gpu ids
gpu_ids = "0"

# the image size
image_size = 512

# the default number of source images, it will be updated if the actual number of sources <= num_source
num_source = 2

# the assets directory. This is very important, please download it from `one_drive_url` firstly.
assets_dir = "/content/iPERCore/assets"

# the output directory.
output_dir = "./results"

# the model id of this case. This is a random model name.
# model_id = "model_" + str(time.time())

# # This is a specific model name, and it will be used if you do not change it.
# model_id = "axing_1"

# symlink from the actual assets directory to this current directory
work_asserts_dir = os.path.join("./assets")
if not os.path.exists(work_asserts_dir):
    os.symlink(osp.abspath(assets_dir), osp.abspath(work_asserts_dir),
               target_is_directory=(platform.system() == "Windows"))

cfg_path = osp.join(work_asserts_dir, "configs", "deploy.toml")

In [ ]:
%cd /content/iPERCore
!rm -rf images
!mkdir images


In [ ]:
# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "front_only" 

# the source input information, here \" is escape character of double duote "
src_path = "\"path?=/content/iPERCore/images/1_2EE292AB-228F-43A1-BE25-086973036C77.jpg,name?=front_only\""

ref_path = "\"path?=/content/iPERCore/assets/samples/references/23.072.mp4,"  \
             "name?=23.072," \
             "pose_fc?=300\""
#%cd /content/iPERCore/images
#front_img_full_path = os.path.join("/content/iPERCore/images", front_img[0])
#src_path = "\"path?=" + front_img_full_path + ",name?=front_only\""

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id    \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path

In [ ]:
mp4 = open("./results/primitives/front_only/synthesis/imitations/front_only-23.072.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")